# 0. what for
1. 计算input data
# 1. Preparations
## 1.1 全局设置

In [20]:
import numpy as np
#import cupy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from GT_model.GT_2.SA_modified import SABoltzmann
from sko.tools import set_run_mode
from GT_model.GT_2.SA_for_PT_funcs_delta_eq1 import *

# large data
data_np_path = r'E:\DATA\large_dta\auctions_np.csv'
settings_np_path = r'E:\DATA\large_dta\settings_np.csv'

# output
params_opitim_path = r'E:\DATA\large_dta\params_opitim.csv'
GT_1_large_P = r'E:\DATA\large_dta\GT_1_large_P.csv'
GT_2_large_P = r'E:\DATA\large_dta\GT_2_large_P.csv'

In [15]:
# For large data
data = pd.read_csv(data_np_path, encoding="utf-8")
data_key = pd.read_csv(settings_np_path, encoding="utf-8")
# 有`N_uniq_auction`组setting
N_uniq_auction= data_key.shape[0]
data_key.head(),data.head()

(                                 item     k     y  retail
 0                     20 BIDS VOUCHER  0.01  0.01    14.0
 1                     20 BIDS VOUCHER  0.01  0.01    15.0
 2                     50 BIDS VOUCHER  0.01  0.01    35.0
 3                50 FREEBIDS VOUCHER!  0.01  0.01    35.0
 4  MAKITA CORDLESS ARTICULATED WRENCH  0.01  0.01    59.0,
               item     k     y  retail     N         P  cnt_N
 0  20 BIDS VOUCHER  0.01  0.01    14.0  18.0  0.026525     10
 1  20 BIDS VOUCHER  0.01  0.01    14.0  18.0  0.026525     10
 2  20 BIDS VOUCHER  0.01  0.01    14.0  15.0  0.039788     15
 3  20 BIDS VOUCHER  0.01  0.01    14.0  32.0  0.015915      6
 4  20 BIDS VOUCHER  0.01  0.01    14.0  16.0  0.015915      6)

# 2. GT-1

In [5]:
# whether set threshold or not
threshold_flag = 1
plot_flag = 0

K = 300
col_names = ['bidincrement','bidfee','retail']
tmp = np.arange(0,K)
tmp_str = [str(x) for x in tmp]
col_names.extend(tmp_str)

P = pd.DataFrame(columns=col_names)

print("Done")

Done


In [6]:
######################################################### new
# for every uniq setting i
cnt = 0
for i in tqdm(range(0,N_uniq_auction)):

    # Get params
    v = float(data_key.loc[i,'retail'])            # retail price = valuation
    d = float(data_key.loc[i,'k'])                    # bid increment
    b = float(data_key.loc[i,'y'])                   # bid fee
    T_i = int((v-b)/d)                              # duration limitation

    # solve for U
    U = [0] * (T_i + 1)                               # the prob. that someone offers a bid in t_th round
    U[0] = 1
    for t in range(1,len(U)):
        U[t] = 1.0-(b/(v-d*(t-1)))
        assert U[t]>=0, "U[t]<0 when t ={},and b = {},v = {}, d = {}".format(t,b,v,d)

    # decide whether set threshold or not
    if (threshold_flag != 1):
        K = T_i

    # solve for P
    p = np.array([0.0]*(K))
    tmp = np.array([0.0]*(K+3))                         # tmp的大小不需要太精确
    tmp[0] = 1.0

    # solve for P
    if((K+1) > T_i):                                    # threshold>T_i时，补零
        cnt += 1
        for t in range(0,T_i):
            tmp[t+1] = tmp[t]*U[t]                      # tmp[t]存了到(t-1)的连乘积
            p[t] = (1-U[t+1])*tmp[t+1]
            print(p[t])
        assert np.min(p) >= 0, "A: in {}, p<0, when b = {},v = {}, d = {}".format(i,b,v,d)

    elif((K+1) == T_i):                                 # threshold==T_i时
        for t in range(0,K):
            tmp[t+1] = tmp[t]*U[t]                      # tmp[t]存了到(t-1)的连乘积
            p[t] = (1-U[t+1])*tmp[t+1]
        p[K-1] = 1-sum(p)
        assert np.min(p) >= 0, "B: in {}, p<0, when b = {},v = {}, d = {}".format(i,b,v,d)

    elif((K+1) < T_i):                              # threshold<T_i时，归一化
        for t in range(0,K):
            tmp[t+1] = tmp[t]*U[t]                      # tmp[t]存了到(t-1)的连乘积
            p[t] = (1-U[t+1])*tmp[t+1]
        assert np.min(p) >= 0, "C: in {}, p<0, when b = {},v = {}, d = {}".format(i,b,v,d)

        # 归一化
        scale_sum = np.sum(p)
        p = p/scale_sum

    else:
        print("Undecided yet. Waiting to be completed.")

    # concat df P
    pd_tmp = pd.DataFrame(data=[[d,b,v]])
    pd_tmp = pd.concat([pd_tmp,pd.DataFrame(p).T],axis=1)
    pd_tmp.columns = col_names
    P = pd.concat([P,pd_tmp],ignore_index=True)


print("Done")
print(cnt)

  0%|          | 0/78 [00:00<?, ?it/s]

Done
0


In [8]:
P.to_csv(GT_1_large_P,encoding="utf-8",index=False,header=True)

# 3. GT-2
## 3.1 SA
0. 基础设置

In [21]:
# for SA
# initial params
table_5_M = [0.025,3.72]
# lower/ upper bound
lb = [-0.3,0.01]
ub = [0.3,18]

1. 根据key提取数据

In [22]:
# For large datadset
def get_large_data_i(i):
        return data[(data['item'] == data_key.loc[i,"item"]) & (data['k'] == data_key.loc[i,"k"]) & (data['y'] == data_key.loc[i,"y"])].copy()

2. loss function

In [23]:
def loss_func(params,other_params):
    # start_time_loss = datetime.datetime.now()
    alpha = params[0]
    # delta = 1
    labda = params[1]
    # max_T,v,d,b,T_i,cnt_n_2_i = other_params
    NLL=0.0

    # For all settings, sum up their loss
    for i in range(0,N_uniq_auction):

        # Extract data with same setting
        data_i = get_large_data_i(i)
        data_i.reset_index(drop=True,inplace=True)

        # Get params
        v = float(data_key.loc[i,'retail'])            # retail price = valuation
        d = float(data_key.loc[i,'k'])                    # bid increment
        b = float(data_key.loc[i,'y'])                   # bid fee

        T_i = data_i['N'].astype(int)                   # auction duration sequence
        max_T = int(max(T_i))                           # max duration value

        assert max_T <= (v-b)/d, "max_T is longer than T in theory!"

        flag = 1
        # Solve for U from Equi. condt.
        U_i = [0] * (max_T + 1)
        U_i[0] = 1

        for t in range(1,max_T+1):
            U_i[t] = f_Equi(t, v, d, b, alpha, labda)
            if(flag & (U_i[t]<=0)):
                # print(f"t:{t} -- u starts to <= 0.0: {U_i[t]} -- alpha : {alpha} -- lamda : {labda}")
                flag = 0

        # Calculate NLL under this setting & PT params in order to optim. the PT params
        nll = 0.0
        if(U_i[0]==1):
            U_i.pop(0)
        for idx in range(0,data_i.shape[0]):
            # sum up the log prob among all durations of this auction
            # nll1 += np.cumprod(U_i)[T_i[idx]-2]*U_i[T_i[idx]-1]* cnt_n_2_i[idx]
            # NLL1 == nll
            nll += ( np.sum( np.log( U_i[0:(T_i[idx]-1)] ) ) + np.log(1-U_i[(T_i[idx]-1)]) )
        # print('> The loss costs {time_costs}s \n'.format(time_costs=(datetime.datetime.now() - start_time_loss).total_seconds()))
        NLL += nll
    print(f"NLL loss is {NLL}")
    return float(-NLL)

3. Do SA

In [24]:
params_opitim = pd.DataFrame(columns=['trial_time','alpha','delta','labda','initial_loss','final_loss','avg_loss'])
max_trial = 3

In [ ]:
# Perform SA for all settings at once
for i in range(0,max_trial):
    print(f"> For the *{i}_th* trial")
    print("> Initilizing SA....... \n")
    set_run_mode(loss_func, 'cached')
    set_run_mode(loss_func, 'multithreading')

    initial_t = max(data['cnt_N'].astype(int))        # initial temperature
    # L=30, max_stay_counter=15
    # other_params = [max_T,v,d,b,T_i,cnt_n_2_i]
    sa_boltzmann = SABoltzmann(func=loss_func, x0=table_5_M, other_params = [], T_max=initial_t, T_min=1, learn_rate=0.2, L=20, max_stay_counter=10, lb=lb, ub=ub)

    print("> Now do SA....... \n")
    best_x, best_y = sa_boltzmann.run()
    # print('> The whole inference process costs {time_costs}s \n'.format(time_costs=(datetime.datetime.now() - start_time).total_seconds()))

    print("> SA ENDS....... \n")
    # if(i%10 == 0):
    #     viz.line([0.0]*(sa_boltzmann.iter_cycle+1),[0]*(sa_boltzmann.iter_cycle+1),env = "Loss of PT_2",win = 'Loss of '+str(i),opts= dict(title='Loss of '+str(i)))
    #     viz.line(np.array(sa_boltzmann.generation_best_Y),np.arange(0,sa_boltzmann.iter_cycle+1),env = "Loss of PT_2",win = 'Loss of '+str(i), update='append')

    # append the opitimized params into the df
    df_tmp = pd.DataFrame([ [i,best_x[0],1,best_x[1],sa_boltzmann.generation_best_Y[0],best_y,best_y/data.shape[0]] ],columns=['trial_time','alpha','delta','labda','initial_loss','final_loss','avg_loss'])

    params_opitim = params_opitim.append(df_tmp,ignore_index=True)

> For the *0_th* trial
> Initilizing SA....... 

NLL loss is -78553.83097926383
> Now do SA....... 

NLL loss is -88696.76859425979
NLL loss is -77428.99377801715
NLL loss is -85714.79625466681


C:\Users\Administrator\AppData\Local\Temp\ipykernel_19536\3577596881.py:46: RuntimeWarning: divide by zero encountered in log
  nll += ( np.sum( np.log( U_i[0:(T_i[idx]-1)] ) ) + np.log(1-U_i[(T_i[idx]-1)]) )


NLL loss is -inf
NLL loss is -80382.33026520748
NLL loss is -inf
NLL loss is -80708.06406439615
NLL loss is -78923.12458960124
NLL loss is -79831.86569597317
NLL loss is -79670.04620374723
NLL loss is -79117.98116221881
NLL loss is -inf
NLL loss is -inf
NLL loss is -79943.63948975594
NLL loss is -86135.92314126498
NLL loss is -92215.83852077802
NLL loss is -78934.88905002704
NLL loss is -84210.65010457039
NLL loss is -86019.90988565868
NLL loss is -inf
NLL loss is -115285.47576335217
NLL loss is -78253.0100915832
NLL loss is -77074.87872057279
NLL loss is -82037.78622194812
NLL loss is -137659.18141931266
NLL loss is -inf
NLL loss is -78933.53621000136
NLL loss is -92875.6876502796
NLL loss is -79526.77767902052
NLL loss is -inf
NLL loss is -81684.33565432763
NLL loss is -78790.718262501
NLL loss is -121073.60589017473
NLL loss is -inf
NLL loss is -84666.07591085305
NLL loss is -87077.0451943527
NLL loss is -84867.99527822413
NLL loss is -inf
NLL loss is -inf
NLL loss is -80935.7726845

C:\Users\Administrator\AppData\Local\Temp\ipykernel_19536\446937815.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  params_opitim = params_opitim.append(df_tmp,ignore_index=True)


NLL loss is -78553.83097926383
> Now do SA....... 

NLL loss is -89220.97734009892
NLL loss is -80178.4283218221


C:\Users\Administrator\AppData\Local\Temp\ipykernel_19536\3577596881.py:46: RuntimeWarning: divide by zero encountered in log
  nll += ( np.sum( np.log( U_i[0:(T_i[idx]-1)] ) ) + np.log(1-U_i[(T_i[idx]-1)]) )


NLL loss is -inf
NLL loss is -92887.76505672807
NLL loss is -84088.36554503461
NLL loss is -80143.70847201982
NLL loss is -78782.38806342524
NLL loss is -80943.10325957513
NLL loss is -inf
NLL loss is -100469.14158870913
NLL loss is -77783.13155119117
NLL loss is -inf
NLL loss is -inf
NLL loss is -115205.7889543876
NLL loss is -89187.18341743163
NLL loss is -inf
NLL loss is -inf
NLL loss is -inf
NLL loss is -82922.93362531575
NLL loss is -78185.58624773353
NLL loss is -86303.72817032022
NLL loss is -113296.79147273203
NLL loss is -80689.21407198993
NLL loss is -89858.55274462876
NLL loss is -85866.78704833711
NLL loss is -136700.62562680864
NLL loss is -79961.64997601691
NLL loss is -84645.61151835164
NLL loss is -95214.69135874619
NLL loss is -79508.94312165742
NLL loss is -88606.99131037063
NLL loss is -80238.51036249427
NLL loss is -78763.23093344206
NLL loss is -82016.47433868654
NLL loss is -inf
NLL loss is -80317.68151499955
NLL loss is -95734.34140439068
NLL loss is -129557.8112

In [ ]:
# Save 'params_opitim' for later check
params_opitim.to_csv(params_opitim_path, header=True, encoding="utf-8",index=False)

## 3.2 Generate